In [112]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus, CSVClassificationCorpus
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

In [113]:
from flair.data import Sentence

In [114]:
path = './resume2/'
model = SequenceTagger.load(path + 'best-model.pt')

2022-08-15 13:33:29,866 loading file ./resume2/best-model.pt
2022-08-15 13:33:33,404 SequenceTagger predicts: Dictionary with 35 tags: O, S-geo, B-geo, E-geo, I-geo, S-tim, B-tim, E-tim, I-tim, S-org, B-org, E-org, I-org, S-per, B-per, E-per, I-per, S-gpe, B-gpe, E-gpe, I-gpe, S-art, B-art, E-art, I-art, S-eve, B-eve, E-eve, I-eve, S-nat, B-nat, E-nat, I-nat, <START>, <STOP>


In [117]:
model.modules

<bound method Module.modules of SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05, inplace=False)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4196, out_features=4196, bias=True)
  (rnn): LSTM(4196, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=35,

In [105]:
# define columns
columns = {0: 'text', 1: 'ner'}

# this is the folder in which train, test and dev files reside
data_folder = 'utf8/'

# init a corpus using column format, data folder and the names of the train, dev and test files
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file=None,
                              test_file='ner_test_utf8.txt',
                              dev_file=None,
                              skip_first_line=True)

2022-08-13 05:57:34,575 Reading data from utf8
2022-08-13 05:57:34,575 Train: utf8/ner_train_utf8.txt
2022-08-13 05:57:34,576 Dev: None
2022-08-13 05:57:34,578 Test: utf8/ner_test_utf8.txt


In [106]:
s = corpus.get_all_sentences()[0]
s.get_label('ner').value

'geo'

In [107]:
result =  model.evaluate(corpus.test, mini_batch_size=32, out_path=f"{path}predictions.txt", gold_label_type='ner')
print(result)

100%|██████████| 145/145 [23:46<00:00,  9.84s/it]

2022-08-13 06:21:36,853 Evaluating as a multi-label problem: False

Results:
- F-score (micro) 0.8514
- F-score (macro) 0.6598
- Accuracy 0.7887

By class:
              precision    recall  f1-score   support

         geo     0.8495    0.9062    0.8769      3562
         tim     0.8995    0.8657    0.8823      1996
         org     0.7728    0.7134    0.7419      2031
         gpe     0.9594    0.9528    0.9561      1611
         per     0.8115    0.8003    0.8059      1582
         art     0.2105    0.0952    0.1311        42
         eve     0.4545    0.3704    0.4082        27
         nat     0.8333    0.3333    0.4762        15

   micro avg     0.8539    0.8490    0.8514     10866
   macro avg     0.7239    0.6297    0.6598     10866
weighted avg     0.8516    0.8490    0.8495     10866

Loss: 0.06043708324432373'
